# 데이터 파일 결합 및 전처리

In [8]:
import pandas as pd
import re, glob, os, math, html
from pathlib import Path


# ---------- 1. 파일 로드 & 병합 ---------- #
file_paths = [
    "./data/naver_blog_보험설계사 2주 합격.csv",
    "./data/naver_blog_보험설계사 준비 이유.csv",
    "./data/naver_blog_보험설계사 합격 후기.csv",
    "./data/보험설계사 불합격 후기.csv",
    "./data/보험설계사 시험 꿀팁.csv",
    "./data/보험설계사 자격증.csv"
]

frames = []
for path in file_paths:
    try:
        df = pd.read_csv(path, encoding="utf-8")
    except UnicodeDecodeError:
        df = pd.read_csv(path, encoding="cp949")
    df["source_file"] = Path(path).name
    frames.append(df)

df = pd.concat(frames, ignore_index=True)

df

,author_id,title,link,content,source_file
0,이지웨이,[직장인투잡] 보험설계사 시험 2주 만에 합격한 공부비법!(원더 스마트 플래너 ),https://blog.naver.com/easywayforu/223641816981,"저는 \n""원더 스마트 플래너 앱""\n으로 투잡을 시작한 직장인입니다.\n10월에 ...",naver_blog_보험설계사 2주 합격.csv
1,봉봉,"보험설계사 시험 2주 준비 / 합격 후기 /투잡성공후기 /손해보험,제3보험,생명보험...",https://blog.naver.com/bblove6042/223744321253,"몇년 전 부터 보험에 관심이 있었던 터라... \n사실 보험 설계사는 싫었고, (회...",naver_blog_보험설계사 2주 합격.csv
2,cherish,2주만에 보험설계사 합격하는 공부법 +25만원,https://blog.naver.com/cherishyy_/223715753626,안녕하세요\n오늘은 2주만 투자해서 보험설계사 합격하고\n25만원까지 받는 공부법을...,naver_blog_보험설계사 2주 합격.csv
3,깝녕,손해보험 설계사 시험 합격과정,https://blog.naver.com/bobba_/223881156811,보빠(보험에 빠지다)\n첫글을 쓰기에 앞서 필자는 올해 33살이다. 지금부터 쓰는 ...,naver_blog_보험설계사 2주 합격.csv
4,야간열차,손해보험 설계사 자격시험 보려면? 절차·과목·합격 기준,https://blog.naver.com/gurugram/223892159706,"요즘 직장을 다니면서도 부업을 찾거나, 인생 2막을 준비하려는 분들 정말 많으시죠....",naver_blog_보험설계사 2주 합격.csv
...,...,...,...,...,...
415,지독사,■#지독사 손해보험설계사 자격증시험 합격!! ㅋ,https://blog.naver.com/dongbantwojobs/22382233...,지독사 본업은\n공기좋은 곳에서 \n관리자로 있어요~~ㅎ\n​\n공기좋은 곳에서\...,보험설계사 자격증.csv
416,라니,보험 설계사 인식 왜 엇갈릴까? 보험설계사자격증 준비 과정은?,https://blog.naver.com/dldprth8852/223740309656,보험설계사자격증 취득이 손쉬울까요? \n보험설계사 인식은 왜 엇갈릴까요? \n이번 ...,보험설계사 자격증.csv
417,공부하는 언니,손해 보험 설계사 자격증 도움 될까?,https://blog.naver.com/inni_healing/223856554286,​\n4월 14일 나는 손해보험 설계사 자격증을 땄다.\n​\n왜 보험설계사를 하려...,보험설계사 자격증.csv
418,흥굿보험흥팀장,"보험설계사 자격증 시험 접수하는 방법, 공부 및 합격 꿀팁",https://blog.naver.com/rlfntm4/223740494681,안녕하세요. \n박흥윤 팀장\n입니다.\n오늘은 \n보험 영업의 첫 걸음\n이라고 ...,보험설계사 자격증.csv


In [95]:
# ---------- 2. 콘텐츠 컬럼 식별 & 정제 ---------- #
# 'content', '본문', '내용' 중 존재하는 컬럼 선택
content_col = None
for cand in ["content", "본문", "내용", "텍스트", "text"]:
    if cand in df.columns:
        content_col = cand
        break

if content_col is None:
    raise ValueError("본문/내용 컬럼을 찾지 못했습니다.")

def clean_text(t):
    if pd.isna(t):
        return ""
    t = html.unescape(str(t))
    # HTML 태그 제거
    t = re.sub(r"<[^>]+>", " ", t)
    # 공백 정리
    t = re.sub(r"\s+", " ", t)
    return t.strip()

df["clean_text"] = df[content_col].apply(clean_text)

In [96]:
# ---------- 3. 부업 여부(business) ---------- #

# 부업이면 1 아니면 0
business_pattern = re.compile(r"(부업|투잡|n[ -]?잡|side[\s-]?job|사이드잡)", re.I)
df["business"] = df["clean_text"].str.contains(business_pattern).astype(int)

C:\Users\Admin\AppData\Local\Temp\ipykernel_13408\1459861299.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["business"] = df["clean_text"].str.contains(business_pattern).astype(int)


In [97]:
# ---------- 4. 준비 기간 추출 & 빠른 준비(quick) ---------- #
# (\d+)\s*(주|개월|일) 패턴 → 일수 환산
def extract_days(text):
    matches = re.findall(r"(\d+)\s*(주|개월|일)", text)
    days_list = []
    for num, unit in matches:
        n = int(num)
        if unit == "일":
            days_list.append(n)
        elif unit == "주":
            days_list.append(n * 7)
        elif unit == "개월":
            days_list.append(n * 30)
    return min(days_list) if days_list else math.nan

df["prep_days"] = df["clean_text"].apply(extract_days)
df["quick"] = df["prep_days"].apply(lambda x: 1 if not math.isnan(x) and x <= 14 else 0)   # 2주 이하면 1 아니면 0

In [98]:
df

,author_id,title,link,content,source_file,clean_text,business,prep_days,quick
0,이지웨이,[직장인투잡] 보험설계사 시험 2주 만에 합격한 공부비법!(원더 스마트 플래너 ),https://blog.naver.com/easywayforu/223641816981,"저는 \n""원더 스마트 플래너 앱""\n으로 투잡을 시작한 직장인입니다.\n10월에 ...",naver_blog_보험설계사 2주 합격.csv,"저는 ""원더 스마트 플래너 앱"" 으로 투잡을 시작한 직장인입니다. 10월에 처음 원...",1,14.0,1
1,봉봉,"보험설계사 시험 2주 준비 / 합격 후기 /투잡성공후기 /손해보험,제3보험,생명보험...",https://blog.naver.com/bblove6042/223744321253,"몇년 전 부터 보험에 관심이 있었던 터라... \n사실 보험 설계사는 싫었고, (회...",naver_blog_보험설계사 2주 합격.csv,"몇년 전 부터 보험에 관심이 있었던 터라... 사실 보험 설계사는 싫었고, (회사를...",1,3.0,1
2,cherish,2주만에 보험설계사 합격하는 공부법 +25만원,https://blog.naver.com/cherishyy_/223715753626,안녕하세요\n오늘은 2주만 투자해서 보험설계사 합격하고\n25만원까지 받는 공부법을...,naver_blog_보험설계사 2주 합격.csv,안녕하세요 오늘은 2주만 투자해서 보험설계사 합격하고 25만원까지 받는 공부법을 소...,0,14.0,1
3,깝녕,손해보험 설계사 시험 합격과정,https://blog.naver.com/bobba_/223881156811,보빠(보험에 빠지다)\n첫글을 쓰기에 앞서 필자는 올해 33살이다. 지금부터 쓰는 ...,naver_blog_보험설계사 2주 합격.csv,보빠(보험에 빠지다) 첫글을 쓰기에 앞서 필자는 올해 33살이다. 지금부터 쓰는 글...,0,5.0,1
4,야간열차,손해보험 설계사 자격시험 보려면? 절차·과목·합격 기준,https://blog.naver.com/gurugram/223892159706,"요즘 직장을 다니면서도 부업을 찾거나, 인생 2막을 준비하려는 분들 정말 많으시죠....",naver_blog_보험설계사 2주 합격.csv,"요즘 직장을 다니면서도 부업을 찾거나, 인생 2막을 준비하려는 분들 정말 많으시죠....",1,14.0,1
...,...,...,...,...,...,...,...,...,...
415,지독사,■#지독사 손해보험설계사 자격증시험 합격!! ㅋ,https://blog.naver.com/dongbantwojobs/22382233...,지독사 본업은\n공기좋은 곳에서 \n관리자로 있어요~~ㅎ\n​\n공기좋은 곳에서\...,보험설계사 자격증.csv,지독사 본업은 공기좋은 곳에서 관리자로 있어요~~ㅎ ​ 공기좋은 곳에서 일한다는 것...,1,4.0,1
416,라니,보험 설계사 인식 왜 엇갈릴까? 보험설계사자격증 준비 과정은?,https://blog.naver.com/dldprth8852/223740309656,보험설계사자격증 취득이 손쉬울까요? \n보험설계사 인식은 왜 엇갈릴까요? \n이번 ...,보험설계사 자격증.csv,보험설계사자격증 취득이 손쉬울까요? 보험설계사 인식은 왜 엇갈릴까요? 이번 글에서 ...,0,NaN,0
417,공부하는 언니,손해 보험 설계사 자격증 도움 될까?,https://blog.naver.com/inni_healing/223856554286,​\n4월 14일 나는 손해보험 설계사 자격증을 땄다.\n​\n왜 보험설계사를 하려...,보험설계사 자격증.csv,​ 4월 14일 나는 손해보험 설계사 자격증을 땄다. ​ 왜 보험설계사를 하려고 하...,0,14.0,1
418,흥굿보험흥팀장,"보험설계사 자격증 시험 접수하는 방법, 공부 및 합격 꿀팁",https://blog.naver.com/rlfntm4/223740494681,안녕하세요. \n박흥윤 팀장\n입니다.\n오늘은 \n보험 영업의 첫 걸음\n이라고 ...,보험설계사 자격증.csv,안녕하세요. 박흥윤 팀장 입니다. 오늘은 보험 영업의 첫 걸음 이라고 할 수 있는 ...,0,NaN,0


In [99]:
# ---------- 5. 기본 통계 & 교차표 ---------- #
summary_counts = df[["business", "quick"]].agg(['sum', 'mean']).T
summary_counts.rename(columns={'sum': '합계(건)', 'mean': '비율'}, inplace=True)

crosstab_bq = pd.crosstab(df["business"], df["quick"], margins=True)
crosstab_bq.index = ['일반', '부업', '총합']
crosstab_bq.columns = ['일반 준비', '단기 준비', '총합']

In [100]:
# ---------- 6. 결과 출력 / 시각화 ---------- #
print("=== 핵심 통계 ===")
summary_counts

=== 핵심 통계 ===


,합계(건),비율
business,118.0,0.280952
quick,228.0,0.542857


In [101]:
print("\n=== 부업 여부 × 단기 준비 교차표 ===")
crosstab_bq


=== 부업 여부 × 단기 준비 교차표 ===


,일반 준비,단기 준비,총합
일반,140,162,302
부업,52,66,118
총합,192,228,420


In [102]:
new_df = df[['author_id','title','link','clean_text','business','prep_days','quick']]
new_df

,author_id,title,link,clean_text,business,prep_days,quick
0,이지웨이,[직장인투잡] 보험설계사 시험 2주 만에 합격한 공부비법!(원더 스마트 플래너 ),https://blog.naver.com/easywayforu/223641816981,"저는 ""원더 스마트 플래너 앱"" 으로 투잡을 시작한 직장인입니다. 10월에 처음 원...",1,14.0,1
1,봉봉,"보험설계사 시험 2주 준비 / 합격 후기 /투잡성공후기 /손해보험,제3보험,생명보험...",https://blog.naver.com/bblove6042/223744321253,"몇년 전 부터 보험에 관심이 있었던 터라... 사실 보험 설계사는 싫었고, (회사를...",1,3.0,1
2,cherish,2주만에 보험설계사 합격하는 공부법 +25만원,https://blog.naver.com/cherishyy_/223715753626,안녕하세요 오늘은 2주만 투자해서 보험설계사 합격하고 25만원까지 받는 공부법을 소...,0,14.0,1
3,깝녕,손해보험 설계사 시험 합격과정,https://blog.naver.com/bobba_/223881156811,보빠(보험에 빠지다) 첫글을 쓰기에 앞서 필자는 올해 33살이다. 지금부터 쓰는 글...,0,5.0,1
4,야간열차,손해보험 설계사 자격시험 보려면? 절차·과목·합격 기준,https://blog.naver.com/gurugram/223892159706,"요즘 직장을 다니면서도 부업을 찾거나, 인생 2막을 준비하려는 분들 정말 많으시죠....",1,14.0,1
...,...,...,...,...,...,...,...
415,지독사,■#지독사 손해보험설계사 자격증시험 합격!! ㅋ,https://blog.naver.com/dongbantwojobs/22382233...,지독사 본업은 공기좋은 곳에서 관리자로 있어요~~ㅎ ​ 공기좋은 곳에서 일한다는 것...,1,4.0,1
416,라니,보험 설계사 인식 왜 엇갈릴까? 보험설계사자격증 준비 과정은?,https://blog.naver.com/dldprth8852/223740309656,보험설계사자격증 취득이 손쉬울까요? 보험설계사 인식은 왜 엇갈릴까요? 이번 글에서 ...,0,NaN,0
417,공부하는 언니,손해 보험 설계사 자격증 도움 될까?,https://blog.naver.com/inni_healing/223856554286,​ 4월 14일 나는 손해보험 설계사 자격증을 땄다. ​ 왜 보험설계사를 하려고 하...,0,14.0,1
418,흥굿보험흥팀장,"보험설계사 자격증 시험 접수하는 방법, 공부 및 합격 꿀팁",https://blog.naver.com/rlfntm4/223740494681,안녕하세요. 박흥윤 팀장 입니다. 오늘은 보험 영업의 첫 걸음 이라고 할 수 있는 ...,0,NaN,0


In [103]:
new_df.to_csv('cleaned_text_all.csv', )

In [104]:
data = pd.read_csv('./cleaned_text_all.csv')

In [105]:
data

,Unnamed: 0,author_id,title,link,clean_text,business,prep_days,quick
0,0,이지웨이,[직장인투잡] 보험설계사 시험 2주 만에 합격한 공부비법!(원더 스마트 플래너 ),https://blog.naver.com/easywayforu/223641816981,"저는 ""원더 스마트 플래너 앱"" 으로 투잡을 시작한 직장인입니다. 10월에 처음 원...",1,14.0,1
1,1,봉봉,"보험설계사 시험 2주 준비 / 합격 후기 /투잡성공후기 /손해보험,제3보험,생명보험...",https://blog.naver.com/bblove6042/223744321253,"몇년 전 부터 보험에 관심이 있었던 터라... 사실 보험 설계사는 싫었고, (회사를...",1,3.0,1
2,2,cherish,2주만에 보험설계사 합격하는 공부법 +25만원,https://blog.naver.com/cherishyy_/223715753626,안녕하세요 오늘은 2주만 투자해서 보험설계사 합격하고 25만원까지 받는 공부법을 소...,0,14.0,1
3,3,깝녕,손해보험 설계사 시험 합격과정,https://blog.naver.com/bobba_/223881156811,보빠(보험에 빠지다) 첫글을 쓰기에 앞서 필자는 올해 33살이다. 지금부터 쓰는 글...,0,5.0,1
4,4,야간열차,손해보험 설계사 자격시험 보려면? 절차·과목·합격 기준,https://blog.naver.com/gurugram/223892159706,"요즘 직장을 다니면서도 부업을 찾거나, 인생 2막을 준비하려는 분들 정말 많으시죠....",1,14.0,1
...,...,...,...,...,...,...,...,...
415,415,지독사,■#지독사 손해보험설계사 자격증시험 합격!! ㅋ,https://blog.naver.com/dongbantwojobs/22382233...,지독사 본업은 공기좋은 곳에서 관리자로 있어요~~ㅎ ​ 공기좋은 곳에서 일한다는 것...,1,4.0,1
416,416,라니,보험 설계사 인식 왜 엇갈릴까? 보험설계사자격증 준비 과정은?,https://blog.naver.com/dldprth8852/223740309656,보험설계사자격증 취득이 손쉬울까요? 보험설계사 인식은 왜 엇갈릴까요? 이번 글에서 ...,0,NaN,0
417,417,공부하는 언니,손해 보험 설계사 자격증 도움 될까?,https://blog.naver.com/inni_healing/223856554286,​ 4월 14일 나는 손해보험 설계사 자격증을 땄다. ​ 왜 보험설계사를 하려고 하...,0,14.0,1
418,418,흥굿보험흥팀장,"보험설계사 자격증 시험 접수하는 방법, 공부 및 합격 꿀팁",https://blog.naver.com/rlfntm4/223740494681,안녕하세요. 박흥윤 팀장 입니다. 오늘은 보험 영업의 첫 걸음 이라고 할 수 있는 ...,0,NaN,0


# 본격적으로 분석 시작

In [106]:
pd.DataFrame(data['clean_text'])

,clean_text
0,"저는 ""원더 스마트 플래너 앱"" 으로 투잡을 시작한 직장인입니다. 10월에 처음 원..."
1,"몇년 전 부터 보험에 관심이 있었던 터라... 사실 보험 설계사는 싫었고, (회사를..."
2,안녕하세요 오늘은 2주만 투자해서 보험설계사 합격하고 25만원까지 받는 공부법을 소...
3,보빠(보험에 빠지다) 첫글을 쓰기에 앞서 필자는 올해 33살이다. 지금부터 쓰는 글...
4,"요즘 직장을 다니면서도 부업을 찾거나, 인생 2막을 준비하려는 분들 정말 많으시죠...."
...,...
415,지독사 본업은 공기좋은 곳에서 관리자로 있어요~~ㅎ ​ 공기좋은 곳에서 일한다는 것...
416,보험설계사자격증 취득이 손쉬울까요? 보험설계사 인식은 왜 엇갈릴까요? 이번 글에서 ...
417,​ 4월 14일 나는 손해보험 설계사 자격증을 땄다. ​ 왜 보험설계사를 하려고 하...
418,안녕하세요. 박흥윤 팀장 입니다. 오늘은 보험 영업의 첫 걸음 이라고 할 수 있는 ...


In [108]:

import pandas as pd, numpy as np, umap, hdbscan, faiss
from kss import split_sentences
from umap import UMAP
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ───────────────────────────────────────────────────────
# 1) CSV 한 개 로드 & 문장 단위 분리
# ───────────────────────────────────────────────────────
FILE_PATH = "./cleaned_text_all.csv"
TEXT_COL  = "clean_text"          # ← 열 이름을 맞춰주세요

df = pd.read_csv(FILE_PATH)

sentences = [
    s for text in df[TEXT_COL].dropna().astype(str).tolist()
    for s in split_sentences(text)
]
sent_df = pd.DataFrame({"sent": sentences}).drop_duplicates()

print(f"총 문장 수: {len(sent_df):,}")

# ───────────────────────────────────────────────────────
# 2) KoSBERT 임베딩
# ───────────────────────────────────────────────────────
model = SentenceTransformer("jhgan/ko-sbert-sts")
emb   = model.encode(
    sent_df["sent"].tolist(), batch_size=64, show_progress_bar=True
)

# ───────────────────────────────────────────────────────
# 3) “어려움” 시드 문장 지정
# ───────────────────────────────────────────────────────
seed_texts = [
    "시험이 너무 어려워요", "2주 합격이 가능할까요?", "탈락했어요", "어떻게 공부해야할지 모르겠어요", "시험이 막막해요"
]
seed_emb = model.encode(seed_texts)

# 시드-전체 유사도 계산
sim = cosine_similarity(seed_emb, emb).max(axis=0)
sent_df["sim_to_seed"] = sim

hard_df = sent_df[sent_df["sim_to_seed"] >= 0.45].copy()
print(f"‘어려움’ 후보 문장 수: {len(hard_df):,}")

# ───────────────────────────────────────────────────────
# 4) 후보 문장 세부 군집화
# ───────────────────────────────────────────────────────

um = UMAP(n_neighbors=10, n_components=5, random_state=42)\
        .fit_transform(emb[hard_df.index])
    
import hdbscan
labels = hdbscan.HDBSCAN(min_cluster_size=25).fit_predict(um)
hard_df["cluster"] = labels

# ───────────────────────────────────────────────────────
# 5) 결과 요약 출력
# ───────────────────────────────────────────────────────
summary = (
    hard_df.groupby("cluster")
           .agg(
               count=("sent", "size"),
               examples=("sent", lambda x: x.iloc[:5].tolist())
           )
           .sort_values("count", ascending=False)
)

pd.set_option("display.max_colwidth", None)
print(summary.head(10))   # 상위 10개 클러스터 미리보기

# 필요하면 CSV 저장
summary.to_csv("hardness_clusters_summary.csv", encoding="utf-8-sig")

ImportError: cannot import name 'UMAP' from 'umap' (C:\Users\Admin\miniforge3\envs\ai_serving\Lib\site-packages\umap\__init__.py)

# 구글 제미나이 분류

### 보험설계사 자격증 취득 이유

In [4]:
import os
import pandas as pd
import time
from tqdm.auto import tqdm
import google.generativeai as genai

#──────────────────────────────────────────────
# 0) 설정 ‑‑ CSV 경로 / 텍스트 열 / 저장 경로
#──────────────────────────────────────────────
CSV_PATH      = "./cleaned_text_all.csv"
TEXT_COL      = "clean_text"          # 원문 열
NEW_COL       = "Gemini_동기"         # 새로 만들 열
SAVE_PATH     = "./cleaned_with_motives.csv"

#──────────────────────────────────────────────
# 1) Gemini 초기화
#──────────────────────────────────────────────
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise EnvironmentError("환경변수 GEMINI_API_KEY 를 먼저 설정하세요!")

genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.0-flash")

# 프롬프트 템플릿: 분류 라벨 정의
SYSTEM_PROMPT = """
너는 보험설계사(보험 모집인) 자격증을 준비하는 사람들이 남긴 글을 읽고,
그들이 자격증을 취득하려는 “주된 동기”를 한 단어/한글 라벨로 분류한다.

가능한 라벨:
- 부업: 본업 외 'N잡', 용돈·부수입·단기수익 목적
- 전업: 보험 영업을 ‘주 직업·커리어’로 삼고 싶은 경우
- 자기계발: 공부, 지식 확장, 금융·투자 이해 향상 등 비금전적 이유
- 급전: 당장 생활비·빚 등 긴급한 돈벌이 필요
- 기타: 위 범주에 명확히 안 맞는 경우

반드시 다음 JSON 형식으로만 출력:
{ "label": "부업" }
"""

#──────────────────────────────────────────────
# 2) 데이터 불러오기
#──────────────────────────────────────────────
df = pd.read_csv(CSV_PATH)
df = df[[TEXT_COL]].dropna().reset_index(drop=True)

# 이미 분류된 행은 건너뛰도록 준비
if NEW_COL in df.columns:
    todo_idx = df[df[NEW_COL].isna()].index
else:
    df[NEW_COL] = pd.NA
    todo_idx = df.index

#──────────────────────────────────────────────
# 3) 한 행씩 Gemini‑pro 호출 → 동기 라벨 추출
#──────────────────────────────────────────────
def classify_motive(text: str) -> str:
    prompt = SYSTEM_PROMPT + f"\n\n### 글:\n{text[:1500]}"  # 토큰 절약(최대 1500자)
    try:
        res = model.generate_content(prompt, safety_settings={"harassment": "block_none"})
        label = res.text.strip()
        # JSON 파싱
        import json, re
        label = json.loads(re.search(r"\{.*\}", label, re.S).group())["label"]
        return label
    except Exception as e:
        return pd.NA

for i in tqdm(todo_idx, desc="Gemini 분류 중"):
    df.at[i, NEW_COL] = classify_motive(df.at[i, TEXT_COL])
    # 과금/쿼터 보호: 1 req/sec
    time.sleep(1.2)

#──────────────────────────────────────────────
# 4) 결과 저장
#──────────────────────────────────────────────
df.to_csv(SAVE_PATH, index=False)
print(f"✅ 저장 완료 → {SAVE_PATH}")


Gemini 분류 중: 100%|██████████████████████████████████████████████████████████████████████████████████████| 420/420 [10:03<00:00,  1.44s/it]

✅ 저장 완료 → ./cleaned_with_motives.csv


### 시험 공부하는 과정에서 겪는 어려움 -> 다시 돌리기

In [3]:
# ─── 0) 라이브러리 ──────────────────────────────
import os, re, time, json, csv, hashlib, shelve
import pandas as pd
from tqdm.auto import tqdm
import google.generativeai as genai
from google.api_core.exceptions import ResourceExhausted

CSV_IN   = "./data/보험설계사 불합격 후기.csv"
TEXT_COL = "content"
CSV_OUT  = "./exam_hard_reason_tagged.csv"
CACHE_DB = "./gemini_cache.db"          # shelve 형식

# ─── 1) Gemini 세팅 ────────────────────────────
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel("gemini-2.0-flash")
RATE_SEC = 4.2                           # 분당 15회 이하

SYSTEM = """
You are an assistant that classifies **multiple** short Korean sentences.
For each sentence, return ONE of:

① 내용-이해 ② 암기-부담 ③ 시간-부족 ④ 학습-자료 ⑤ 동기-저하 ⑥ 기타

Respond **JSON array**:
[
  {"tag":"암기-부담", "reason":"요약 근거"},
  ...
]
"""

# ─── 2) 데이터 준비 ─────────────────────────────
df = pd.read_csv(CSV_IN)
def split_sentences(t): return [s.strip() for s in re.split(r"[.!?]\s+", str(t)) if s.strip()]
sentences = [s for t in df[TEXT_COL].dropna() for s in split_sentences(t)]

HARD_PAT = r"(어렵|힘들|지루|포기|시간|부담|몰라|공부|암기|교재)"
hard_sents = [s for s in sentences if re.search(HARD_PAT, s)]
print(f"🔍 어려움 후보 문장 개수: {len(hard_sents):,}")

# ─── 3) 캐시 DB 열기 ────────────────────────────
cache = shelve.open(CACHE_DB)

def gemini_bulk_tag(sent_batch):
    """5~7 문장 단위로 태깅 요청 (캐시 활용)"""
    key = hashlib.md5("\n".join(sent_batch).encode()).hexdigest()
    if key in cache:
        return cache[key]
    prompt = SYSTEM + "\n\n" + "\n".join(f"- {s}" for s in sent_batch)
    while True:
        try:
            resp = model.generate_content(prompt).text
            data = json.loads(resp)
            cache[key] = data
            time.sleep(RATE_SEC)
            return data
        except ResourceExhausted as e:
            wait = getattr(e, "retry_delay", None)
            sec  = wait.seconds if wait else 20
            print(f"⚠️ 쿼터 초과, {sec}s 대기 후 재시도"); time.sleep(sec)
        except Exception as e:
            print("⚠️ 파싱 오류:", e); return [{"tag":"error","reason":str(e)}]*len(sent_batch)

# ─── 4) 배치 처리 ──────────────────────────────
BATCH = 6
out_rows = []
for i in tqdm(range(0, len(hard_sents), BATCH)):
    batch = hard_sents[i:i+BATCH]
    results = gemini_bulk_tag(batch)
    for s, r in zip(batch, results):
        out_rows.append({"sentence": s,
                         "difficulty_tag": r.get("tag",""),
                         "why": r.get("reason","")})

cache.close()

# ─── 5) 저장 ──────────────────────────────────
pd.DataFrame(out_rows)\
  .to_csv(CSV_OUT, index=False, quoting=csv.QUOTE_MINIMAL, encoding="utf-8-sig")
print("✅ 완료! →", CSV_OUT)

🔍 어려움 후보 문장 개수: 580


  0%|                                                                                           | 0/97 [00:00<?, ?it/s]

⚠️ 쿼터 초과, 20s 대기 후 재시도
⚠️ 쿼터 초과, 20s 대기 후 재시도


  0%|                                                                                           | 0/97 [00:40<?, ?it/s]


KeyboardInterrupt: 

In [115]:
motivate_df['Gemini_동기'].value_counts()

Gemini_동기
부업      72
전업      64
기타      33
자기계발    27
급전       4
Name: count, dtype: int64

In [124]:
import pandas as pd

# 1) 컬럼 폭 제한 해제  ──────────────────────────────
pd.set_option("display.max_colwidth", None)   # None = 무제한

# ‘부업’ 동기 문장 전체 확인
view_df = motivate_df[motivate_df["Gemini_동기"] == "자기계발"]
display(view_df[["clean_text", "Gemini_동기"]].head(30))   # 원하는 행수만큼

# 2) 또는 to_string()으로 한꺼번에 출력 ─────────────
print(
    motivate_df[motivate_df["Gemini_동기"] == "자기계발"]
    [["clean_text", "Gemini_동기"]]
    .to_string(index=False)
)

clean_text  \
14                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

### 사용자의 니즈 파악

In [2]:
# ───────────────────────────────────────────────
# 0) 라이브러리 & 환경 설정
# ───────────────────────────────────────────────
import os, time, json, hashlib, random
import pandas as pd
from tqdm.auto import tqdm
import google.generativeai as genai
from google.api_core.exceptions import ResourceExhausted

CSV_PATH  = "./cleaned_with_motives_utf.csv"     # 원본 데이터
TEXT_COL  = "clean_text"                         # 본문 열
LABEL_COL = "Gemini_동기"                        # 부업/전업 라벨 열
OUT_PATH  = "./needs_with_examples.csv"          # 최종 저장 파일

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel("gemini-2.0-flash")

# ───────────────────────────────────────────────
# 유틸
# ───────────────────────────────────────────────
def chunk_texts(texts, max_chars=3500):
    buf, chunks = "", []
    for t in texts:
        if len(buf) + len(t) + 1 > max_chars:
            chunks.append(buf); buf = t + "\n"
        else:
            buf += t + "\n"
    if buf: chunks.append(buf)
    return chunks

def gemini_call(prompt, cache, retry_sleep=40):
    h = hashlib.md5(prompt.encode()).hexdigest()
    if h in cache:                         # 캐시 hit
        return cache[h]
    while True:
        try:
            txt = model.generate_content(prompt).text.strip()
            cache[h] = txt
            return txt
        except ResourceExhausted as e:     # 429 대기·재시도
            wait = getattr(e, "retry_delay", None)
            sec  = wait.seconds if wait else retry_sleep
            print(f"🔄 429… {sec}s 대기 후 재시도"); time.sleep(sec)

# ───────────────────────────────────────────────
# 1) 데이터 로드 & 그룹 순회
# ───────────────────────────────────────────────
df         = pd.read_csv(CSV_PATH)
cache      = {}
MAX_SAMPLE = 1500          # 그룹별 최대 문장 수
rows       = []            # ← 여기 담아서 DataFrame 생성

for label, gdf in df.groupby(LABEL_COL):
    print(f"\n=== {label} 그룹 처리 ===")
    sents  = gdf[TEXT_COL].dropna().unique().tolist()[:MAX_SAMPLE]
    random.shuffle(sents)                               # 근거 다양성 확보
    chunks = chunk_texts(sents, 3500)

    # ── 1차: 니즈 & 예시 추출 ──────────────────────────
    need_ex_list = []
    for chunk in tqdm(chunks, desc=f"[{label}] 1st pass"):
        prompt = f"""
        ‘보험설계사 자격증 준비/활동’ 중 {label} 사용자들의 실제 발화입니다.
        각 문장을 꼼꼼히 읽고, **핵심 Pain Point·니즈** 3~5개를 뽑은 뒤
        • "니즈": (한글 3~5어절)
        • "예시": 해당 니즈를 잘 보여주는 원문 문장 1~2개
        를 JSON 배열로 출력하세요.  예시:

        [
          {{"need": "빠른 합격 방법 정보 부족", "example": "시험 일정이 자꾸 바뀌는데 어디서 확인하나요?"}},
          ...
        ]

        ---------- 텍스트 ----------
        {chunk}
        ---------------------------
        """
        resp = gemini_call(prompt, cache)
        try:
            need_ex_list.extend(json.loads(resp))
        except Exception:
            # LLM JSON 오류 대비 → 무시
            continue

    # ── 2차: 중복·유사 니즈 통합 요약 ──────────────────
    merge_prompt = f"""
    아래 JSON 배열은 {label} 그룹 사용자들의 니즈-예시 초안(중복 포함)입니다.
    유사 항목을 묶어 **3~6개**로 압축하고, 예시는 가장 대표 문장으로 1개만 남겨
    다시 JSON 배열로 출력하세요.

    {json.dumps(need_ex_list, ensure_ascii=False)}
    """
    merged = gemini_call(merge_prompt, cache)
    try:
        final_list = json.loads(merged)
    except Exception:
        # JSON 파싱 실패 시 임시 처리
        final_list = need_ex_list[:6]

    # → rows 에 저장
    for item in final_list:
        rows.append({
            "그룹"    : label,
            "니즈"    : item.get("need", "").strip(),
            "근거예시": item.get("example", "").strip()
        })

# ───────────────────────────────────────────────
# 2) CSV 저장
# ───────────────────────────────────────────────
out_df = pd.DataFrame(rows).drop_duplicates()
out_df.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
print("\n✅ 완료! →", OUT_PATH)



=== 급전 그룹 처리 ===


[급전] 1st pass: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  2.32s/it]



=== 기타 그룹 처리 ===


[기타] 1st pass:  40%|██████████████████████████████████▊                                                    | 10/25 [00:19<00:30,  2.01s/it]

🔄 429… 40s 대기 후 재시도


...

🔄 429… 40s 대기 후 재시도


[전업] 1st pass:  88%|████████████████████████████████████████████████████████████████████████████▉          | 46/52 [03:02<00:12,  2.02s/it]

🔄 429… 40s 대기 후 재시도


[전업] 1st pass: 100%|███████████████████████████████████████████████████████████████████████████████████████| 52/52 [03:57<00:00,  4.57s/it]



✅ 완료! → ./needs_with_examples.csv


### 원더와 life with 후기 블로그 글을 분석하여 인사이트 도출해내기

In [2]:
# ───────────────────────────────────────────────
# 0. 라이브러리
# ───────────────────────────────────────────────
import pandas as pd
import re, os, json, time, hashlib
import google.generativeai as genai
from tqdm.auto import tqdm
from google.api_core.exceptions import ResourceExhausted

# ───────────────────────────────────────────────
# 1. 데이터 로딩 (CSV 파일 경로)
# ───────────────────────────────────────────────
wonder_path = './data/원더_보험_후기.csv'
hanhwa_path = './data/한화_life_with.csv'

wonder_df = pd.read_csv(wonder_path, encoding="utf-8-sig")
hanhwa_df = pd.read_csv(hanhwa_path, encoding="utf-8-sig")

# 두 데이터 병합
df = pd.concat([wonder_df, hanhwa_df], ignore_index=True)

# ───────────────────────────────────────────────
# 2. 문장 분리 및 필터링 ('어려움' 키워드)
# ───────────────────────────────────────────────
TEXT_COL = 'content'  # CSV에서 본문 열 이름 확인!

def split_sentences(text):
    return [s.strip() for s in re.split(r'[.!?]\s+', str(text)) if s.strip()]

df_sentences = pd.DataFrame({
    'sentence': [sentence for txt in df[TEXT_COL].dropna() for sentence in split_sentences(txt)]
}).drop_duplicates().reset_index(drop=True)

# 키워드로 필터링
HARD_PAT = r"(어렵|힘들|포기|지루|부담|모르|시간|준비|공부|암기|교재|자료)"
hard_df = df_sentences[df_sentences['sentence'].str.contains(HARD_PAT)].reset_index(drop=True)

print(f'🔍 어려움 후보 문장 개수: {len(hard_df)}')

# ───────────────────────────────────────────────
# 3. Gemini API 설정
# ───────────────────────────────────────────────
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel("gemini-2.0-flash")

# 어려움 원인 카테고리 예시
CATEGORIES = [
    "정보-부족",
    "학습자료-부족",
    "시간-부족",
    "동기/의지-저하",
    "암기/내용-부담",
    "제도/절차-불만",
    "기타"
]

SYSTEM_PROMPT = (
    "당신은 보험 자격증 시험 준비 과정에서 느끼는 어려움 원인을 분석합니다. "
    "다음 문장을 읽고 주요 어려움 원인을 다음 중 하나로 분류한 후, 간단히(20자 이내) 이유를 설명하세요:\n"
    + "\n".join(f"- {cat}" for cat in CATEGORIES) +
    "\nJSON 형식으로 응답하세요: {\"cause\":\"카테고리\",\"why\":\"이유\"}."
)

# ───────────────────────────────────────────────
# 4. Gemini 호출 함수 (캐싱, 재시도 로직 포함)
# ───────────────────────────────────────────────
cache = {}

def gemini_tag(sentence):
    hash_key = hashlib.md5(sentence.encode()).hexdigest()
    if hash_key in cache:
        return cache[hash_key]

    prompt = f"{SYSTEM_PROMPT}\n\n문장: \"{sentence}\"\n응답 JSON:"
    retry_delay = 5
    max_retries = 5
    retries = 0

    while retries < max_retries:
        try:
            rsp = model.generate_content(prompt).text.strip()
            data = json.loads(rsp)
            result = (data.get("cause", "기타"), data.get("why", ""))
            cache[hash_key] = result
            return result
        except ResourceExhausted:
            print(f"⚠️ API 쿼터 초과, {retry_delay}초 후 재시도")
            time.sleep(retry_delay)
            retry_delay *= 2
            retries += 1
        except Exception:
            result = ("기타", "파싱 실패")
            cache[hash_key] = result
            return result

    result = ("기타", "API 재시도 초과")
    cache[hash_key] = result
    return result

# ───────────────────────────────────────────────
# 5. 배치로 Gemini 호출 및 결과 저장
# ───────────────────────────────────────────────
results_cause, results_why = [], []

for sentence in tqdm(hard_df['sentence'], desc='Gemini 분석 진행'):
    cause, why = gemini_tag(sentence)
    results_cause.append(cause)
    results_why.append(why)
    time.sleep(1)  # 호출 간격 설정 (API 보호)

# 결과 데이터프레임 저장
hard_df['어려움_원인'] = results_cause
hard_df['AI_설명'] = results_why

# CSV로 결과 저장
result_path = './자격증_어려움_원인분석.csv'
hard_df.to_csv(result_path, index=False, encoding='utf-8-sig')

print(f'✅ 분석 완료! 저장 위치: {result_path}')

# 어려움 원인 분포 확인
print("\n📊 원인별 분포 현황")
print(hard_df['어려움_원인'].value_counts())


C:\Users\Admin\miniforge3\envs\fintech\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Admin\AppData\Local\Temp\ipykernel_21412\544612926.py:36: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  hard_df = df_sentences[df_sentences['sentence'].str.contains(HARD_PAT)].reset_index(drop=True)


🔍 어려움 후보 문장 개수: 774


Gemini 분석 진행:   0%|▏                                                               | 2/774 [00:03<24:41,  1.92s/it]

⚠️ API 쿼터 초과, 5초 후 재시도
⚠️ API 쿼터 초과, 10초 후 재시도
⚠️ API 쿼터 초과, 20초 후 재시도
⚠️ API 쿼터 초과, 40초 후 재시도


Gemini 분석 진행:   0%|▏                                                             | 2/774 [01:19<8:32:56, 39.87s/it]


KeyboardInterrupt: 

In [9]:
wonder_path = './data/원더_보험_후기.csv'
hanhwa_path = './data/한화_life_with.csv'

wonder_df = pd.read_csv(wonder_path, encoding="utf-8-sig")
hanhwa_df = pd.read_csv(hanhwa_path, encoding="utf-8-sig")

# ---------- 2. 콘텐츠 컬럼 식별 & 정제 ---------- #
# 'content', '본문', '내용' 중 존재하는 컬럼 선택
content_col = None
for cand in ["content", "본문", "내용", "텍스트", "text"]:
    if cand in df.columns:
        content_col = cand
        break

if content_col is None:
    raise ValueError("본문/내용 컬럼을 찾지 못했습니다.")

def clean_text(t):
    if pd.isna(t):
        return ""
    t = html.unescape(str(t))
    # HTML 태그 제거
    t = re.sub(r"<[^>]+>", " ", t)
    # 공백 정리
    t = re.sub(r"\s+", " ", t)
    return t.strip()

wonder_df["clean_text"] = wonder_df[content_col].apply(clean_text)
hanhwa_df["clean_text"] = hanhwa_df[content_col].apply(clean_text)

In [12]:
wonder_df['clean_text'][0]

"공부하고 25만원 벌기 개꿀아님? 개꿀은 아닐 수 있음... 난 경고했다... ~ 실제 입금내역 첨부 ~ 손해보험설계사 시험후기, 합격썰 작년에 한창 재테크 유튜버 영상 찾아볼때 온갖 부업은 다 따라해봤다. 그 중 괜찮은건 거의 없었다. '고수익 부업은 다 사기야~ 세상에 쉽게 돈 버는 법은 없다!' 라고 생각하고 있었는데, 솔깃한 내용을 발견했다. 손해보험 설계사 자격시험 합격하고, 롯데손해보험에서 운영하는 '원더 앱'에 설계사로 등록만 해도 25만원을 준다는 이야기였다. 사실 보험에 0.1%도 관심없고, 가입할 생각도 없지만 속는셈치고 따라해봤다. \u200b 첫 번째 스텝은 원더 앱 깔고 손해보험 설계사 시험대비 동영상을 시청해야했다. 그냥 영상 노트북에 켜두고 열심히 딴짓했다. 그리고 시험 이틀전이 됐다. 모의고사를 풀어봤다. 망했다. 생각보다 문제가 너무 어려웠다. 나는 운전면허시험 수준일 줄 알았는데, 보험에 대한 상식이 어느정도 필요했다. TMI지만 필자는 광고업에 종사한 적이 있으며, 광고주중 하나가 OO손해보험이었다. 그래서 나름 일반인보단 손해보험 상식이 있다고 생각했다. 근데 모의고사 점수가 처참하다니 자존심이 상했다. 그래서 이틀동안 진짜 미친듯이 모의고사 돌려가며 오답정리를 하기 시작했다. 아무튼 결론은 합격엔딩이었지만, 공부 아예 안하고 날로 먹는 시험은 아니라는 점~ 특히 보험 상식 아예없는 사람이면 공부를 하긴 해야한다. 이것도 공부해두면 살아가는데 어딘가 도움은 되겠지~라고 정신 승리하면서 하니까 나름 할만은 했다! \u200b \u200b \u200b 돈 받았니? 그럼 이제 해촉하고 말소를 해보자 합격하고 나면 원더 앱에서 알람이 온다. 설계사 등록을 하고 어쩌구 저쩌구~ 하면서 롯데손해보험에 설계사로 정식 등록을 하게된다. 그러면 25만 포인트가 쌓여 출금할 수 있게 된다. 다만 합격하고 이 돈을 받는 과정이 약 한 달은 걸렸던것 같다. 그리고 본인을 포함해 보험을 가입하면 더 돈을 받을 수 있다. 그치만 나는 체리피커니까 